# 🧹 AgentCore 엔드투엔드 정리

이 노트북은 AgentCore 엔드투엔드 튜토리얼 중에 생성된 모든 리소스에 대한 포괄적인 정리 프로세스를 제공합니다.

## 개요

이 정리 프로세스는 다음을 제거합니다:
- **메모리**: AgentCore Memory 리소스 및 저장된 데이터
- **런타임**: Agent 런타임 인스턴스 및 ECR 리포지토리
- **보안**: 실행 역할 및 권한 부여 공급자 리소스
- **관측성**: CloudWatch 로그 그룹 및 스트림
- **로컬 파일**: 생성된 구성 및 코드 파일

⚠️ **중요**: 이 정리는 되돌릴 수 없습니다. 계속하기 전에 중요한 데이터(필요한 경우)를 저장했는지 확인하십시오.

---

## 단계 1: 필요한 종속성 가져오기

정리 프로세스에 필요한 모든 모듈과 도우미 함수를 로드합니다.

In [ ]:
import json

from lab_helpers.lab2_memory import REGION
from lab_helpers.utils import (
    delete_agentcore_runtime_execution_role,
    delete_ssm_parameter,
    cleanup_cognito_resources,
    get_customer_support_secret,
    delete_customer_support_secret,
    agentcore_memory_cleanup,
    gateway_target_cleanup,
    runtime_resource_cleanup,
    delete_observability_resources,
    local_file_cleanup,
    get_ssm_parameter,
    policy_engine_cleanup
)

print("✅ Dependencies imported successfully")
print(f"🌍 Working in region: {REGION}")

## 단계 2: 메모리 리소스 정리하기

AgentCore Memory 리소스 및 관련 데이터를 제거합니다.

In [ ]:
print("🧠 Starting Memory cleanup...")
agentcore_memory_cleanup(get_ssm_parameter("/app/customersupport/agentcore/memory_id"))

## 단계 3: 런타임 리소스 정리하기

AgentCore 런타임, ECR 리포지토리 및 관련 AWS 리소스를 제거합니다.

In [ ]:
print("🚀 Starting Runtime cleanup...")
runtime_resource_cleanup(
    get_ssm_parameter("/app/customersupport/agentcore/runtime_arn")
)

## 단계 4: 게이트웨이 리소스 정리하기
타겟, 게이트웨이를 제거합니다.

In [ ]:
print("⚙️ Starting Policy Engine Cleanup...")
policy_engine_cleanup(get_ssm_parameter("/app/customersupport/agentcore/policy_engine_id"))

print("⚙️ Starting Gateway Cleanup...")
gateway_target_cleanup(get_ssm_parameter("/app/customersupport/agentcore/gateway_id"))

## 단계 5: 보안 리소스 정리하기

실행 역할 및 인증 리소스를 제거합니다.

In [ ]:
print("🛡️  Starting Security cleanup...")
try:
    # bedrock_client = boto3.client("bedrock", region_name=REGION)

    # Delete execution role
    print("  🗑️  Deleting AgentCore Runtime execution role...")
    delete_agentcore_runtime_execution_role()
    print("  ✅ Execution role deleted")

    # Delete SSM parameter
    print("  🗑️  Deleting SSM parameter...")
    delete_ssm_parameter("/app/customersupport/agentcore/runtime_arn")
    print("  ✅ SSM parameter deleted")

    # Clean up Cognito and secrets
    print("  🗑️  Cleaning up Cognito resources...")
    cs = json.loads(get_customer_support_secret())
    cleanup_cognito_resources(cs["pool_id"])
    print("  ✅ Cognito resources cleaned up")

    print("  🗑️  Deleting customer support secret...")
    delete_customer_support_secret()
    print("  ✅ Customer support secret deleted")

except Exception as e:
    print(f"  ⚠️  Error during security cleanup: {e}")

## 단계 6: 로컬 파일 정리하기

로컬 디렉터리에서 생성된 구성 및 코드 파일을 제거합니다.

In [ ]:
print("📁 Starting Local Files cleanup...")
local_file_cleanup()

## 단계 7: 관측성 리소스 정리하기

에이전트 모니터링에 사용되는 CloudWatch 로그 그룹 및 스트림을 제거합니다.

In [ ]:
print("📊 Starting Observability cleanup...")

delete_observability_resources()

## 🎉 정리 완료!

모든 AgentCore 리소스가 정리되었습니다. 제거된 항목에 대한 요약은 다음과 같습니다:

In [ ]:
print("\n" + "=" * 60)
print("🧹 CLEANUP COMPLETED SUCCESSFULLY! 🧹")
print("=" * 60)
print()
print("📋 Resources cleaned up:")
print("  🧠 Memory: AgentCore Memory resources and data")
print("  🚀 Runtime: Agent runtime and ECR repository")
print("  🛡️ Security: Roles, and SSM secrets")
print("  📊 Observability: CloudWatch logs")
print("  📁 Files: Local configuration files")
print()
print("✨ Your AWS account is now clean and ready for new experiments!")
print("\nThank you for completing the AgentCore End-to-End tutorial! 🚀")